

## Estructura del Proyecto
- **Entrada**: Diccionario PDF bilingüe (Quechua-Español / Español-Quechua)
- **Salida**: Archivos JSON estructurados + librería Python para consultas

## Tareas
1. Extracción de texto crudo del PDF
2. Identificación y separación de secciones
3. Diseño e implementación de parsers
4. Generación de archivos JSON estructurados
5. Desarrollo de librería de utilidades
6. Validación y testing

In [1]:
# Instalación de librerías necesarias
# !pip install PyMuPDF pdfplumber regex

# Importaciones
import fitz  
import pdfplumber
import re
import time
import json
import os
from typing import List, Dict, Optional, Tuple
from collections import defaultdict
import pandas as pd

print("Librerías instaladas correctamente")

Librerías instaladas correctamente


In [9]:
# TAREA 1 y 2: Extracción de texto del PDF y separación de secciones

def extraer_texto_pdf(ruta_pdf: str) -> str:
    """
    Extrae texto crudo del PDF preservando estructura básica sin alteraciones
    """
    texto_completo = ""
    
    try:
        # Usar PyMuPDF para extracción
        doc = fitz.open(ruta_pdf)
        
        for pagina_num in range(doc.page_count):
            pagina = doc[pagina_num]
            texto = pagina.get_text()
            texto_completo += texto
        
        doc.close()
        
    except Exception as e:
        print(f"Error con PyMuPDF: {e}")
        
        # Alternativa con pdfplumber
        try:
            with pdfplumber.open(ruta_pdf) as pdf:
                for pagina in pdf.pages:
                    texto = pagina.extract_text()
                    if texto:
                        texto_completo += texto
        except Exception as e2:
            print(f"Error con pdfplumber: {e2}")
            return ""
    
    return texto_completo

def guardar_texto_crudo(texto: str, ruta_salida: str = "diccionario_raw.txt"):
    """
    Guarda el texto crudo extraído del PDF
    """
    with open(ruta_salida, 'w', encoding='utf-8') as f:
        f.write(texto)
    print(f"Texto crudo guardado en: {ruta_salida}")

# Ejecutar extracción
ruta_pdf = "diccionario-qeswa-academia-mayor.pdf"
if os.path.exists(ruta_pdf):
    print("Extrayendo texto del PDF...")
    texto_crudo = extraer_texto_pdf(ruta_pdf)
    guardar_texto_crudo(texto_crudo)
    print(f"Texto extraído: {len(texto_crudo)} caracteres")
else:
    print(f"Archivo PDF no encontrado: {ruta_pdf}")
    print("Por favor, asegúrese de que el archivo esté en el directorio correcto")

Extrayendo texto del PDF...
Texto crudo guardado en: diccionario_raw.txt
Texto extraído: 1930001 caracteres


In [10]:
# TAREA 3: Extracción y limpieza de secciones del diccionario

def limpiar_encabezados_y_patrones(texto: str) -> str:
    """
    Limpia el texto eliminando encabezados de página, títulos, letras de sección y otros patrones no deseados.
    """
    lineas = texto.split('\n')
    lineas_limpias = []
    
    for linea in lineas:
        linea_original = linea
        linea = linea.strip()
        
        # Ignorar líneas vacías
        if not linea:
            continue
        
        # PATRONES A ELIMINAR:
        
        # 1. Números de página junto con DICCIONARIO
        if re.match(r'^\d+$', linea) or 'DICCIONARIO' in linea.upper():
            continue
        
        # 2. Símbolos decorativos como ◄●►
        if re.match(r'^[◄●►\-=\*\+\s]+$', linea):
            continue
        
        # 3. Título SIMI TAQE
        if 'SIMI TAQE' in linea.upper():
            continue
        
        # 4. Letras sueltas de sección (una sola letra mayúscula en su propia línea)
        if re.match(r'^[A-Z]$', linea):
            continue
        
        # 5. Encabezados típicos de PDF
        if any(patron in linea.upper() for patron in [
            'PÁGINA', 'CAPÍTULO', 'SECCIÓN', 'ÍNDICE', 'CONTENIDO',
            'DICCIONARIO QUECHUA', 'ESPAÑOL - QUECHUA', 'QUECHUA - ESPAÑOL'
        ]):
            continue
        
        # 6. Líneas que son solo números o símbolos
        if re.match(r'^[\d\s\-\.\*\+◄●►]+$', linea):
            continue
        
        # 7. Líneas muy cortas que no parecen ser contenido útil 
        if len(linea) < 3:
            continue
        
        # 8. Patrones específicos de encabezados de página
        if re.match(r'^\d+\s*◄●►\s*$', linea) or re.match(r'^◄●►\s*\d+\s*$', linea):
            continue
        
        # Si la línea pasó todos los filtros, mantenerla
        lineas_limpias.append(linea_original)
    
    return '\n'.join(lineas_limpias)

def separar_secciones_por_lineas(texto: str) -> Tuple[str, str]:
    """
    Separa las secciones usando los números de línea conocidos y aplica limpieza de encabezados
    """
    lineas = texto.split('\n')
    seccion_qe_lineas = lineas[1325:50998]
    
    # Limpiar texto extra al final de la sección Quechua-Español
    seccion_qe_contenido = []
    for linea in seccion_qe_lineas:
        if "ESPAÑOL - QUECHUA" in linea:
            break
        seccion_qe_contenido.append(linea)
    
    seccion_eq_contenido = []
    for i in range(50998, len(lineas)):
        linea = lineas[i].strip()
        if ("DICCIONARIO QUECHUA – ESPAÑOL – QUECHUA" in linea or 
            "se terminó de imprimir" in linea or 
            "talleres gráficos" in linea):
            break
        seccion_eq_contenido.append(lineas[i])
    
    # Aplicar limpieza de encabezados y patrones no deseados
    seccion_qe_texto = '\n'.join(seccion_qe_contenido)
    seccion_eq_texto = '\n'.join(seccion_eq_contenido)
    
    seccion_qe_limpia = limpiar_encabezados_y_patrones(seccion_qe_texto)
    seccion_eq_limpia = limpiar_encabezados_y_patrones(seccion_eq_texto)
    
    return seccion_qe_limpia, seccion_eq_limpia

# Ejecutar extracción y limpieza de secciones
if os.path.exists("diccionario_raw.txt"):
    with open("diccionario_raw.txt", 'r', encoding='utf-8') as f:
        texto_completo = f.read()
    
    print("Procesando secciones del diccionario...")
    
    seccion_qe, seccion_eq = separar_secciones_por_lineas(texto_completo)
    
    # Contar líneas aproximadas para verificar el contenido
    lineas_qe = [l for l in seccion_qe.split('\n') if l.strip()]
    lineas_eq = [l for l in seccion_eq.split('\n') if l.strip()]
    
    print(f"Sección Quechua-Español: {len(lineas_qe)} líneas")
    print(f"Sección Español-Quechua: {len(lineas_eq)} líneas")
    
    # Guardar archivos limpios
    with open("seccion_quechua_espanol.txt", 'w', encoding='utf-8') as f:
        f.write(seccion_qe)
    
    with open("seccion_espanol_quechua.txt", 'w', encoding='utf-8') as f:
        f.write(seccion_eq)
    
    print("Archivos guardados correctamente")
    
else:
    print("Error: archivo diccionario_raw.txt no encontrado")

Procesando secciones del diccionario...
Sección Quechua-Español: 48883 líneas
Sección Español-Quechua: 9264 líneas
Archivos guardados correctamente


In [7]:
# TAREA 4: Diseño e implementación de parsers

def extraer_entradas_completas(texto: str) -> List[str]:
    """
    Extrae lemas completos del texto.
    """
    lineas = texto.split('\n')
    entradas = []
    entrada_actual = ""

    # Patrón que detecta el inicio de una nueva entrada
    # r"((s\.|adj\.|v\.|adv\.|interj\.|prep\.|conj\.|pron\.|alfab\.|loc\.)(\s*y\s*(s\.|adj\.|v\.))?)"
    patron_inicio = (
        r"^([a-zA-ZÀ-ÿñÑ!¡¿?'\-.,–\s]+)[\.!\?]+\s*"
        r"((s\.|adj\.|v\.|adv\.|interj\.|prep\.|conj\.|pron\.|alfab\.|loc\.|p\.|loc\. adv\.)(\s*y\s*(s\.|adj\.|v\.))?)"
        r"(\s*V\.[A-ZÁÉÍÓÚÑ;,\s'!]+)?"
        
    )
    
    for linea in lineas:
        linea = linea.strip()
        if re.match(patron_inicio, linea, re.IGNORECASE):
            if entrada_actual.strip():
                entradas.append(entrada_actual.strip())
            entrada_actual = linea
        else:
            if entrada_actual:
                entrada_actual += " " + linea

    if entrada_actual.strip():
        entradas.append(entrada_actual.strip())

    return entradas

def parsear_entrada(entrada_texto: str, abreviaturas: Dict, es_espanol: bool = False) -> List[Dict]:
    """
    Este módulo implementa toda la lógica para parsear una entrada del diccionario
    y devuelve una lista de diccionarios con la estructura requerida.
    A continuación se enumera los pasos seguidos.
    """
    if len(entrada_texto) < 10:
        return []
    
    # 1. EXTRAER LEMA
    match_lema = re.match(
        r'^([a-zA-ZÀ-ÿñÑ!¡¿?\'\-.,–\s]+?)([\.!\?])\s',
        entrada_texto
    )

    if not match_lema:
        return []
    
    lema = match_lema.group(1).strip()
    signo_final = match_lema.group(2)

    if signo_final in ["!", "?"]:
        lema += signo_final
    
    if ' ' not in lema and not lema[0].isupper():
        lema = lema.lower()
    
    # Extraer resto del texto después del lema
    resto = entrada_texto[len(match_lema.group(0)):].strip()
    
    # Filtro para español: evitar palabras quechuas
    if es_espanol:
        if any(c in lema.lower() for c in ['k', 'w', 'q']) and 'qu' not in lema.lower():
            return []
    
    # Obtener listas de abreviaturas
    categorias_gramaticales = abreviaturas.get('categorias_gramaticales', [])
    campos_semanticos = abreviaturas.get('campos_semanticos', [])
    dialectales = abreviaturas.get('dialectales', [])
    caracteristicas_morfologicas = abreviaturas.get('caracteristicas_morfologicas', [])
    ignorar = abreviaturas.get('ignorar', [])
    
    # 2. EXTRAER CATEGORÍA GRAMATICAL 
    categorias_lista = []
    categoria = ''

    # Buscar primera categoría
    for cat in sorted(categorias_gramaticales, key=len, reverse=True):
        if resto.startswith(cat):
            categoria = cat
            categorias_lista.append(cat)
            resto = resto[len(cat):].strip()
            break

    # Buscar categorías adicionales conectadas con "y"
    if resto.startswith('y '):
        resto = resto[2:].strip()
        for cat in sorted(categorias_gramaticales, key=len, reverse=True):
            if resto.startswith(cat):
                categorias_lista.append(cat)
                resto = resto[len(cat):].strip()
                break

    # si no encuentra categoria, buscar características morfológicas
    caracteristica_encontrada = False
    for cat in sorted(caracteristicas_morfologicas, key=len, reverse=True):
        if resto.startswith(cat):
            caracteristica_encontrada = True
            resto = resto[len(cat):].strip()
            break
    
    if not categoria and not caracteristica_encontrada:
        return []
    
    # 3. DETECTAR TODOS LOS CAMPOS SEMÁNTICOS EN TODA LA DEFINICIÓN
    campos_encontrados = []
    texto_busqueda = resto

    for campo in sorted(campos_semanticos, key=len, reverse=True):
        patron = rf'(?:(?<=^)|(?<=\s)|(?<=\.)){re.escape(campo)}(?=\s|\.|$)'
        
        if re.search(patron, texto_busqueda):
            campos_encontrados.append(campo)

    # Eliminar abreviaturas del campo ignorar
    for abrev in sorted(ignorar, key=len, reverse=True):
        patron = rf'(?:(?<=^)|(?<=\s)|(?<=\.)){re.escape(abrev)}(?=\s|\.|$)'
        texto_busqueda = re.sub(patron, ' ', texto_busqueda, flags=re.IGNORECASE)

    texto_busqueda = re.sub(r'\s+', ' ', texto_busqueda).strip()

    resto = texto_busqueda
    
    # 4. PROCESAMIENTO MEJORADO DE CONTENIDO DESPUES DE BARRAS ||
    if '||' in resto:
        partes_separadas = [a.strip() for a in resto.split('||') if a.strip()]
    else:
        partes_separadas = [resto]
    
    # Estructura unica de entrada que iremos completando
    entrada = {
        'lema': lema,
        'categoria_gramatical': [categoria],
        'campo_semantico': campos_encontrados.copy(),
        'definicion': '',
        'variantes_dialectales': {},
        'sinonimos': [],
        'antonimos': [],  
        'ejemplos': [],
        'remite_a': ''
    }
    
    # Lista para acumular todas las definiciones
    todas_definiciones = []
    
    for i, parte in enumerate(partes_separadas):
        texto = parte.strip()
        
        # 5. DETECTAR REMISIONES TIPO "V. LEMA" 
        match_remision = re.search(r"\bV\.\s+([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑa-záéíóúñ\s'!¡¿?,.-]*?)(?:\.|$)", texto)
        if match_remision:
            remite_a = match_remision.group(1).strip().lower()
            remite_a.rstrip("., ")
            entrada['remite_a'] = remite_a
            texto = re.sub(r"\bV\.\s+[A-ZÁÉÍÓÚÑa-záéíóúñ\s'!¡¿?,.-]+", '', texto).strip()
        
        # 6. DETECTAR CATEGORÍA GRAMATICAL ADICIONAL PARA CASOS COMO "|| adj. insult."
        categoria_adicional = ''
        for cat in sorted(categorias_gramaticales, key=len, reverse=True):
            if texto.startswith(cat):
                categoria_adicional = cat
                if cat not in entrada['categoria_gramatical']:
                    entrada['categoria_gramatical'].append(cat)
                texto = texto[len(cat):].strip()
                break
        
        # 7. DETECTAR CAMPOS SEMÁNTICOS ADICIONALES PARA CASOS COMO "|| Agri."
        campos_adicionales = []
        for campo in sorted(campos_semanticos, key=len, reverse=True):
            if texto.startswith(campo):
                campos_adicionales.append(campo)
                if campo not in entrada['campo_semantico']:
                    entrada['campo_semantico'].append(campo)
                texto = texto[len(campo):].strip()
                break
        
        # 8. DETERMINAR TIPO DE CONTENIDO DESPUÉS DE ||
        es_dialectal = False
        for dialecto in dialectales:
            dialecto_sin_punto = dialecto.rstrip('.')
            if (f'{dialecto_sin_punto}:' in texto or 
                texto.startswith(dialecto_sin_punto + ':') or
                f' {dialecto_sin_punto}:' in texto):
                es_dialectal = True
                break
        
        # 9. EXTRAER EJEMPLOS 
        if 'EJEM:' in texto:
            pos_ejem = texto.find('EJEM:')
            texto_antes = texto[:pos_ejem].strip()
            texto_ejem = texto[pos_ejem + 5:].strip()
            
            # Encontrar final del ejemplo
            fin_ejemplo = len(texto_ejem)
            for dialecto in dialectales:
                dialecto_sin_punto = dialecto.rstrip('.')
                pos_dialecto = texto_ejem.find(f'{dialecto_sin_punto}:')
                if pos_dialecto != -1 and pos_dialecto < fin_ejemplo:
                    fin_ejemplo = pos_dialecto
            
            ejemplo_texto = texto_ejem[:fin_ejemplo].strip(' .,')
            
            # EXTRAER SINÓNIMOS desde el ejemplo
            if 'SINÓN:' in ejemplo_texto:
                pos_sinon_ej = ejemplo_texto.find('SINÓN:')
                ejemplo_limpio = ejemplo_texto[:pos_sinon_ej].strip(' .,')
                sinon_en_ejemplo = ejemplo_texto[pos_sinon_ej + 6:].strip()
                
                # Buscar hasta el próximo punto o final
                fin_sinon_ej = len(sinon_en_ejemplo)
                if '.' in sinon_en_ejemplo:
                    fin_sinon_ej = sinon_en_ejemplo.index('.')
                
                sinon_texto_ej = sinon_en_ejemplo[:fin_sinon_ej].strip(' .,!')
                if sinon_texto_ej:
                    sinonimos_ej = [s.strip() for s in sinon_texto_ej.split(',') if s.strip()]
                    for sin in sinonimos_ej:
                        sin_limpio = re.sub(r'[^\w\sñÑáéíóúÁÉÍÓÚ]', '', sin).strip()
                        if sin_limpio and sin_limpio not in entrada['sinonimos']:
                            entrada['sinonimos'].append(sin_limpio)
                
                ejemplo_texto = ejemplo_limpio
            
            # EXTRAER ANTÓNIMOS desde el ejemplo
            if 'ANTÓN:' in ejemplo_texto:
                pos_anton_ej = ejemplo_texto.find('ANTÓN:')
                ejemplo_limpio = ejemplo_texto[:pos_anton_ej].strip(' .,')
                anton_en_ejemplo = ejemplo_texto[pos_anton_ej + 6:].strip()
                
                # Buscar hasta el próximo punto o final
                fin_anton_ej = len(anton_en_ejemplo)
                if '.' in anton_en_ejemplo:
                    fin_anton_ej = anton_en_ejemplo.index('.')
                
                anton_texto_ej = anton_en_ejemplo[:fin_anton_ej].strip(' .,!')
                if anton_texto_ej:
                    antonimos_ej = [s.strip() for s in anton_texto_ej.split(',') if s.strip()]
                    for ant in antonimos_ej:
                        ant_limpio = re.sub(r'[^\w\sñÑáéíóúÁÉÍÓÚ]', '', ant).strip()
                        if ant_limpio and ant_limpio not in entrada['antonimos']:
                            entrada['antonimos'].append(ant_limpio)
                
                ejemplo_texto = ejemplo_limpio
            
            # Limpiar solo la abreviatura EJEM: pero mantener el texto
            ejemplo_texto = re.sub(r'\s*EJEM:\s*', ' ', ejemplo_texto)
            ejemplo_texto = ejemplo_texto.strip(' .,')
            
            if ejemplo_texto and ejemplo_texto not in entrada['ejemplos']:
                entrada['ejemplos'].append(ejemplo_texto)
            
            # Continuar con el resto después del ejemplo
            resto_ejemplo = texto_ejem[fin_ejemplo:].strip()
            texto = texto_antes + ' ' + resto_ejemplo
            texto = texto.strip()
        
        # 8. EXTRAER SINÓNIMOS 
        if 'SINÓN:' in texto:
            pos_sinon = texto.find('SINÓN:')
            texto_antes = texto[:pos_sinon].strip()
            texto_sinon = texto[pos_sinon + 6:].strip()
            
            fin_sinon = len(texto_sinon)
            
            # Buscar primer punto seguido de mayúscula
            for j, char in enumerate(texto_sinon):
                if char == '.' and j + 1 < len(texto_sinon):
                    siguiente = texto_sinon[j + 1:j + 10].strip()
                    if siguiente and siguiente[0].isupper():
                        fin_sinon = j + 1
                        break
            
            # Buscar abreviaturas dialectales que interrumpen sinónimos
            for dialecto in dialectales:
                dialecto_sin_punto = dialecto.rstrip('.')
                for patron in [f' {dialecto_sin_punto}:', f' {dialecto_sin_punto} ']:
                    pos = texto_sinon.find(patron)
                    if pos != -1 and pos < fin_sinon:
                        fin_sinon = pos
            
            sinon_texto = texto_sinon[:fin_sinon].strip(' .,!')
            if sinon_texto:
                sinonimos = [s.strip() for s in sinon_texto.split(',') if s.strip()]
                entrada['sinonimos'] = [re.sub(r'[^\w\sñÑáéíóúÁÉÍÓÚ]', '', s).strip() 
                                      for s in sinonimos if len(s.strip()) > 1]
            
            # Continuar con el resto después de los sinónimos
            resto_despues_sinon = texto_sinon[fin_sinon:].strip()
            texto = texto_antes + ' ' + resto_despues_sinon
            texto = texto.strip()

        # 9. EXTRAER ANTÓNIMOS  
        if 'ANTÓN:' in texto:
            pos_anton = texto.find('ANTÓN:')
            texto_antes = texto[:pos_anton].strip()
            texto_anton = texto[pos_anton + 6:].strip()
            fin_anton = len(texto_anton)
            for j, char in enumerate(texto_anton):
                if char == '.' and j + 1 < len(texto_anton):
                    siguiente = texto_anton[j + 1:j + 10].strip()
                    if siguiente and siguiente[0].isupper():
                        fin_anton = j + 1
                        break
            for dialecto in dialectales:
                dialecto_sin_punto = dialecto.rstrip('.')
                for patron in [f' {dialecto_sin_punto}:', f' {dialecto_sin_punto} ']:
                    pos = texto_anton.find(patron)
                    if pos != -1 and pos < fin_anton:
                        fin_anton = pos
            anton_texto = texto_anton[:fin_anton].strip(' .,!')
            if anton_texto:
                antonimos = [s.strip() for s in anton_texto.split(',') if s.strip()]
                entrada['antonimos'] = [re.sub(r'[^\w\sñÑáéíóúÁÉÍÓÚ]', '', s).strip() 
                                        for s in antonimos if len(s.strip()) > 1]
            resto_despues_anton = texto_anton[fin_anton:].strip()
            texto = texto_antes + ' ' + resto_despues_anton
            texto = texto.strip()
        
        # 10. EXTRAER VARIANTES DIALECTALES 
        texto_limpio = texto
        dialectales_ordenados = sorted(dialectales, key=len, reverse=True)
        
        for dialecto in dialectales_ordenados:
            dialecto_sin_punto = dialecto.rstrip('.')
            
            # Buscar patrón al inicio o después de espacios
            patrones_dialectal = [
                f'{dialecto_sin_punto}:',      # Al inicio: "Arg:"
                f' {dialecto_sin_punto}:',     # En medio: " Arg:"
                f'^{dialecto_sin_punto}:'      # Exactamente al inicio
            ]
            
            pos_encontrada = -1
            patron_usado = ''
            
            for patron in patrones_dialectal:
                if patron.startswith('^'):
                    # Patrón de inicio exacto
                    if texto_limpio.startswith(patron[1:]):
                        pos_encontrada = 0
                        patron_usado = patron[1:]
                        break
                else:
                    pos = texto_limpio.find(patron)
                    if pos != -1:
                        pos_encontrada = pos
                        patron_usado = patron
                        break
            
            if pos_encontrada != -1:
                texto_antes = texto_limpio[:pos_encontrada].strip()
                texto_despues = texto_limpio[pos_encontrada + len(patron_usado):].strip()
                
                # Encontrar final de variantes dialectales
                fin_dialectal = len(texto_despues)
                
                # Buscar próximo dialecto
                for otro_dialecto in dialectales_ordenados:
                    if otro_dialecto != dialecto:
                        otro_sin_punto = otro_dialecto.rstrip('.')
                        for patron_otro in [f'{otro_sin_punto}:', f' {otro_sin_punto}:']:
                            pos_otro = texto_despues.find(patron_otro)
                            if pos_otro != -1 and pos_otro < fin_dialectal:
                                fin_dialectal = pos_otro
                
                # Buscar marcadores que interrumpen
                for marcador in [' SINÓN:', ' EJEM:', '. SINÓN:', '. EJEM:']:
                    pos_marcador = texto_despues.find(marcador)
                    if pos_marcador != -1 and pos_marcador < fin_dialectal:
                        fin_dialectal = pos_marcador
                
                contenido_dialectal = texto_despues[:fin_dialectal].strip(' .,!')
                
                if contenido_dialectal:
                    regiones_adicionales = []
                    variantes_finales = []
                    
                    # Separar por dos puntos para detectar regiones comprimidas
                    partes_dos_puntos = contenido_dialectal.split(':')
                    
                    if len(partes_dos_puntos) > 1:
                        for parte in partes_dos_puntos[:-1]: 
                            parte = parte.strip()
                            if re.match(r'^[A-Z][a-z]{2,3}$', parte):
                                regiones_adicionales.append(parte)
                        
                        variantes_texto = partes_dos_puntos[-1].strip()
                        variantes_finales = [v.strip() for v in variantes_texto.split(',') if v.strip()]
                    else:
                        variantes_finales = [v.strip() for v in contenido_dialectal.split(',') if v.strip()]
                    
                    # CREAR ESTRUCTURA DE VARIANTES DIALECTALES
                    if '.' in dialecto and len(dialecto.split('.', 1)[1].rstrip('.')) > 0:
                        pais, region_principal = dialecto.split('.', 1)
                        region_principal = region_principal.rstrip('.')
                        
                        if pais not in entrada['variantes_dialectales']:
                            entrada['variantes_dialectales'][pais] = {}
                        
                        # Agregar región principal
                        entrada['variantes_dialectales'][pais][region_principal] = variantes_finales
                        
                        # Agregar regiones adicionales con las mismas variantes
                        for region_adicional in regiones_adicionales:
                            entrada['variantes_dialectales'][pais][region_adicional] = variantes_finales
                            
                    else:
                        pais = dialecto.rstrip('.')
                        entrada['variantes_dialectales'][pais] = variantes_finales
                
                # Limpiar del texto principal
                texto_limpio = texto_antes + ' ' + texto_despues[fin_dialectal:]
                texto_limpio = texto_limpio.strip()
        
        # 10. EXTRAER DEFINICIÓN, LO QUE QUEDA DESPUÉS DE LIMPIAR TODO
        definicion = texto_limpio
        
        # REGLA 1: Limpiar todos los campos semánticos detectados de la definición
        for campo in campos_encontrados:
            # Limpiar campo al inicio de la definición
            definicion = re.sub(f'^{re.escape(campo)}\\s*', '', definicion)
            # Limpiar campo en medio de la definición
            definicion = re.sub(f'\\s*{re.escape(campo)}\\s*', ' ', definicion)
            # Limpiar campo después de punto
            definicion = re.sub(f'\\.\\s*{re.escape(campo)}\\s*', '. ', definicion)
        
        # Limpiar campos adicionales encontrados después de ||
        for campo in campos_adicionales:
            definicion = re.sub(f'^{re.escape(campo)}\\s*', '', definicion)
            definicion = re.sub(f'\\s*{re.escape(campo)}\\s*', ' ', definicion)
        
        # Limpiar marcadores residuales
        definicion = re.sub(r'SINÓN:.*$', '', definicion)
        definicion = re.sub(r'EJEM:.*$', '', definicion)
        definicion = re.sub(r'VARIEDADES:.*$', '', definicion, flags=re.IGNORECASE)
        definicion = re.sub(r"\bV\.\s+[A-ZÁÉÍÓÚÑa-záéíóúñ\s'!¡¿?,.-]+", '', definicion)
        
        # Limpiar dialectales residuales
        for dialecto in dialectales:
            dialecto_sin_punto = dialecto.rstrip('.')
            definicion = re.sub(f'{re.escape(dialecto_sin_punto)}:.*$', '', definicion)
        
        # 12. EXTRAER DEFINICIÓN 
        definicion_parte = texto_limpio
        
        # Limpiar campos semánticos del texto
        for campo in entrada['campo_semantico']:
            definicion_parte = re.sub(f'^{re.escape(campo)}\\s*', '', definicion_parte)
            definicion_parte = re.sub(f'\\s*{re.escape(campo)}\\s*', ' ', definicion_parte)
            definicion_parte = re.sub(f'\\.\\s*{re.escape(campo)}\\s*', '. ', definicion_parte)
        
        # Limpiar marcadores residuales
        definicion_parte = re.sub(r'SINÓN:.*$', '', definicion_parte)
        definicion_parte = re.sub(r'EJEM:.*$', '', definicion_parte)
        definicion_parte = re.sub(r'VARIEDADES:.*$', '', definicion_parte, flags=re.IGNORECASE)
        definicion_parte = re.sub(r'^\([^)]*\)\.\s*', '', definicion_parte)  # Nombres científicos solos
        # Limpiar remisiones residuales
        definicion_parte = re.sub(r"\bV\.\s+[A-ZÁÉÍÓÚÑa-záéíóúñ\s'!¡¿?,.-]+", '', definicion_parte)

        # REGLA 3: Evitar contaminación y detectar nueva entrada
        patron_nueva_entrada = r'\b[a-zA-Z]+\.\s+(s\.|adj\.|v\.|adv\.|interj\.|prep\.|conj\.|pron\.|alfab\.|loc\.|\bV\.)'
        match_nueva = re.search(patron_nueva_entrada, definicion_parte)
        if match_nueva:
            definicion_parte = definicion_parte[:match_nueva.start()].strip()
        
        definicion_parte = re.sub(r'\s+', ' ', definicion_parte).strip(' .,')
        
        # AGREGAR A DEFINICIONES TOTALES
        if i == 0:
            if definicion_parte:
                todas_definiciones.append(definicion_parte)
        else:
            # Partes después de || incluir si hay contenido y no es solo dialectal
            if definicion_parte and not es_dialectal:
                todas_definiciones.append(definicion_parte)
            elif definicion_parte and es_dialectal:
                # Si es dialectal pero hay contenido antes del dialecto, incluirlo
                texto_antes_dialectos = definicion_parte
                for dialecto in dialectales:
                    dialecto_sin_punto = dialecto.rstrip('.')
                    patron_corte = f'{dialecto_sin_punto}:'
                    if patron_corte in texto_antes_dialectos:
                        pos_corte = texto_antes_dialectos.find(patron_corte)
                        texto_antes_dialectos = texto_antes_dialectos[:pos_corte].strip()
                        break
                if texto_antes_dialectos:
                    todas_definiciones.append(texto_antes_dialectos)
        
    # COMBINAR TODAS LAS DEFINICIONES
    if todas_definiciones:
        entrada['definicion'] = '. '.join(todas_definiciones)
    
    
    # FORMATEAR CAMPO SEMÁNTICO 
    if not entrada['campo_semantico']:
        entrada['campo_semantico'] = []
    
    # RETORNAR ENTRADA ÚNICA 
    if (entrada['definicion'] or entrada['sinonimos'] or 
        entrada['variantes_dialectales'] or entrada.get('remite_a')):
        return [entrada]
    else:
        return []


def parsear_seccion_quechua_espanol(texto: str, abreviaturas: Dict[str, List[str]]) -> List[Dict]:
    """
    Parsea la sección Quechua-Español con el parser ultra-estricto
    """
    entradas = []
    entradas_texto = extraer_entradas_completas(texto)
    print(f"Encontradas {len(entradas_texto)} entradas potenciales")
    for entrada_texto in entradas_texto:
        entradas_procesadas = parsear_entrada(
            entrada_texto,
            abreviaturas,
            es_espanol=False
        ) 
        if entradas_procesadas:
            entradas.extend(entradas_procesadas)
    return entradas


def parsear_seccion_espanol_quechua(texto: str, abreviaturas: Dict[str, List[str]]) -> List[Dict]:
    """
    Parsea la sección Español-Quechua con el parser ultra-estricto
    """
    entradas = []
    entradas_texto = extraer_entradas_completas(texto)
    print(f"Encontradas {len(entradas_texto)} entradas potenciales")
    for entrada_texto in entradas_texto:
        entradas_procesadas = parsear_entrada(
            entrada_texto,
            abreviaturas,
            es_espanol=True
        )
        if entradas_procesadas:
            entradas.extend(entradas_procesadas)
    return entradas


print("Parsers implementados correctamente")

Parsers implementados correctamente


In [8]:
# TAREA 5: Generación de archivos JSON estructurados

# Carga las abreviaturas desde el archivo JSON generado anteriormente
def cargar_abreviaturas():
    with open("abreviaturas.json", 'r', encoding='utf-8') as f:
        abreviaturas = json.load(f)
    return abreviaturas



# Ejecuta los parsers y genera los archivos JSON según las especificaciones
def generar_archivos_json():
    abreviaturas = cargar_abreviaturas()
    
    # Procesar sección Quechua-Español
    entradas_qe = []
    if os.path.exists("seccion_quechua_espanol.txt"):
        print("\nProcesando sección Quechua-Español...")
        with open("seccion_quechua_espanol.txt", 'r', encoding='utf-8') as f:
            texto_qe = f.read()
        
        entradas_qe = parsear_seccion_quechua_espanol(texto_qe, abreviaturas)
        print(f"{len(entradas_qe)} entradas procesadas")
    else:
        print("Archivo seccion_quechua_espanol.txt no encontrado")
    
    # Procesar sección Español-Quechua
    entradas_eq = []
    if os.path.exists("seccion_espanol_quechua.txt"):
        print("\nProcesando sección Español-Quechua...")
        with open("seccion_espanol_quechua.txt", 'r', encoding='utf-8') as f:
            texto_eq = f.read()
        
        entradas_eq = parsear_seccion_espanol_quechua(texto_eq, abreviaturas)
        print(f"{len(entradas_eq)} entradas procesadas")
    else:
        print("Archivo seccion_espanol_quechua.txt no encontrado")
    
    # Guardar archivos JSON
    if entradas_qe:
        with open("quechua_espanol.json", 'w', encoding='utf-8') as f:
            json.dump(entradas_qe, f, indent=2, ensure_ascii=False)
        print(f"\nGuardado: quechua_espanol.json")
    
    if entradas_eq:
        with open("espanol_quechua.json", 'w', encoding='utf-8') as f:
            json.dump(entradas_eq, f, indent=2, ensure_ascii=False)
        print(f"Guardado: espanol_quechua.json")


generar_archivos_json()



Procesando sección Quechua-Español...
Encontradas 15232 entradas potenciales
15120 entradas procesadas

Procesando sección Español-Quechua...
Encontradas 5623 entradas potenciales
5547 entradas procesadas

Guardado: quechua_espanol.json
Guardado: espanol_quechua.json


In [2]:
# TAREA 6: Validación y pruebas del diccionario
from diccionario_utils import DiccionarioQuechua
import json

# Instancia global del diccionario
dic = DiccionarioQuechua()

# Normalización de campos para corregir tipo de campos en json
def normalizar_campos(entrada):
    # Normaliza categoria_gramatical y campo_semantico a listas de strings
    for campo in ['categoria_gramatical', 'campo_semantico']:
        valor = entrada.get(campo, None)
        if isinstance(valor, list):
            entrada[campo] = [str(v) for v in valor if isinstance(v, str)]
        elif isinstance(valor, str):
            entrada[campo] = [valor]
        else:
            entrada[campo] = []
    return entrada

# Contar entradas manualmente en archivos JSON
def contar_manual(archivo):
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            datos = json.load(f)
            datos = [normalizar_campos(e) for e in datos]
            return len(datos)
    except:
        return 0

# Validar conteos de entradas
def validar_conteos():
    print("VALIDACIÓN DE CONTEOS DE ENTRADA")
    try:
        manual_qe = contar_manual("quechua_espanol.json")
        manual_eq = contar_manual("espanol_quechua.json")
        lib = dic.contar_entradas()

        print("Quechua-Español")
        print(f"  Entradas manuales en JSON: {manual_qe}")
        print(f"  Entradas reportadas por la librería: {lib['quechua_espanol']}")
        print(f"  Coincidencia: {'Sí' if manual_qe == lib['quechua_espanol'] else 'No'}")
        print()
        print("Español-Quechua")
        print(f"  Entradas manuales en JSON: {manual_eq}")
        print(f"  Entradas reportadas por la librería: {lib['espanol_quechua']}")
        print(f"  Coincidencia: {'Sí' if manual_eq == lib['espanol_quechua'] else 'No'}")
        print()
        if manual_qe != lib['quechua_espanol'] or manual_eq != lib['espanol_quechua']:
            print("Advertencia: Hay diferencia entre el conteo manual y el de la librería. Revise los datos y la función de carga.")
        else:
            print("Los conteos coinciden correctamente.")
    except Exception as e:
        print(f"Error en conteos: {e}")
    print()

# Probar busqueda de lemas
def probar_busquedas():
    print("PRUEBAS DE BÚSQUEDA DE LEMAS")
    lemas = [
        ("achalay", "que"), ("makichay", "que"), ("intuq", "que"), ("kachapuri", "que"),
        ("achuqalla", "que"), ("Eqop", "que"), ("Pachar", "que"),
        ("viajar", "esp"), ("cachorro", "esp"), ("ardid", "esp"),
        ("entrevista", "esp"), ("zurcir", "esp")
    ]
    encontrados = 0
    for lema, tipo in lemas:
        if tipo == "esp":
            res = dic.buscar_por_espanol(lema)
        else:
            res = dic.buscar_por_quechua(lema)
        # Normalizar resultados
        res = [normalizar_campos(e) for e in res]
        if res:
            encontrados += 1
            def_corta = res[0].get('definicion', '')[:60]
            print(f"Lema '{lema}' ({'Español' if tipo == 'esp' else 'Quechua'}): {len(res)} resultado(s). Definición: {def_corta}")
        else:
            print(f"Lema '{lema}' ({'Español' if tipo == 'esp' else 'Quechua'}): sin resultados")
    print(f"Lemas encontrados: {encontrados} de {len(lemas)}")
    print()


# Validar categorías gramaticales
def validar_categorias():
    print("CATEGORÍAS GRAMATICALES PARA QECHUA-ESPAÑOL")
    cats = dic.listar_categorias_gramaticales()
    print(f"Total categorías: {len(cats)}")
    for cat in sorted(cats):
        resultados = dic.buscar_por_categoria_gramatical(cat)
        resultados = [normalizar_campos(e) for e in resultados]
        cnt = len(resultados)
        print(f"{cat}: {cnt}")
    print()


# Validar campos semánticos
def validar_campos():
    print("CAMPOS SEMÁNTICOS")
    campos = dic.listar_campos_semanticos()
    print(f"Total campos: {len(campos)}")
    for campo in sorted(campos):
        resultados = dic.buscar_por_campo_semantico(campo)
        resultados = [normalizar_campos(e) for e in resultados]
        cnt = len(resultados)
        print(f"{campo}: {cnt}")
    print()


# Validar variantes dialectales
def validar_variantes():
    print("VARIANTES DIALECTALES")
    lemas = [
        "achalay", "makichay", "intuq", "kachapuri", "achuqalla", "Eqop", "Pachar",
        "achacha", "achachilla", "achala", "achallqo", "achikyay", "anka", "antipurutu"
    ]
    for lema in lemas:
        variantes = dic.obtener_variantes_dialectales(lema)
        print(f"{lema}: {len(variantes)} variantes")
        if variantes:
            for v in variantes:
                print(f"  {v}")
        else:
            print("  (sin variantes)")
    print()


def validar_sinonimos():
    print("SINÓNIMOS")
    lemas = [
        "achalay", "makichay", "intuq", "kachapuri", "achuqalla", "Eqop", "Pachar",
        "achacha", "achachilla", "achala", "achallqo", "achikyay", "anka", "antipurutu"
    ]
    for lema in lemas:
        res = dic.buscar_por_quechua(lema)
        res = [normalizar_campos(e) for e in res]
        sinonimos = []
        for entrada in res:
            sins = entrada.get('sinonimos', [])
            sinonimos.extend(sins)
        if sinonimos:
            print(f"{lema}: {', '.join(sinonimos)}")
        else:
            print(f"{lema}: (no tiene)")
    print()


def main():
    try:
        validar_conteos()
        probar_busquedas()
        validar_categorias() 
        validar_campos()
        validar_variantes()
        validar_sinonimos()
        
    except Exception as e:
        print(f"Error: {e}")


In [3]:
# Ejecutar validaciones y pruebas
main()

VALIDACIÓN DE CONTEOS DE ENTRADA
Quechua-Español
  Entradas manuales en JSON: 15120
  Entradas reportadas por la librería: 15120
  Coincidencia: Sí

Español-Quechua
  Entradas manuales en JSON: 5547
  Entradas reportadas por la librería: 5547
  Coincidencia: Sí

Los conteos coinciden correctamente.

PRUEBAS DE BÚSQUEDA DE LEMAS
Lema 'achalay' (Quechua): 1 resultado(s). Definición: Ataviar, adornar, acicalar
Lema 'makichay' (Quechua): 1 resultado(s). Definición: Poner las manos o brazos a alguien o algo, en general
Lema 'intuq' (Quechua): 1 resultado(s). Definición: Cercador, sitiador, bloqueador
Lema 'kachapuri' (Quechua): 1 resultado(s). Definición: Enviado, ordenanza, mandadero. Rufián, alcahuete, tercero
Lema 'achuqalla' (Quechua): 1 resultado(s). Definición: (Mustela frenata Lich.) Comadreja. Mamífero mustélido, semip
Lema 'Eqop' (Quechua): 1 resultado(s). Definición: Importante mina de plata en el distrito de Carhuaz, provinci
Lema 'Pachar' (Quechua): 1 resultado(s). Definición: C